In [201]:
#This is a program to predit quality of wines (white+red wines). Here, I did use z score method to remove outliers
import pandas as pd
import numpy as np
white_wine=pd.read_csv('../../practice/winequality-white.csv', sep=';')
red_wine=pd.read_csv('../../practice/winequality-red.csv', sep=';')

In [202]:
# replace space with underline
white_wine.columns=white_wine.columns.str.replace(' ','_')
red_wine.columns=red_wine.columns.str.replace(' ','_')
#add a new column
red_wine['wine_type']='r'
white_wine['wine_type']='w'

In [203]:
#add a new column
red_wine['quality_label']=red_wine['quality'].apply(lambda x: 'low' if x <= 5 else 'medium' if x <= 7 else 'high')
red_wine['quality_label']=pd.Categorical(red_wine['quality_label'], categories=['low','high','medium'])
white_wine['quality_label']=white_wine['quality'].apply(lambda x: 'low' if x <= 5 else 'medium' if x <= 7 else 'high')
white_wine['quality_label']=pd.Categorical(white_wine['quality_label'], categories=['low','high','medium'])

In [204]:
#merge two files
total_wine=pd.concat([white_wine, red_wine])
total_wine= total_wine.set_index(np.arange(len(total_wine)))

In [205]:
# sI use z score method to remove outliers
def outliers_idr(df):
    
    num=df.count(axis=1)
    num_col=num[1]
    
    i=0
    for colu in total_wine.loc[:,'fixed_acidity':'quality']:
        mean_df=df[colu].mean()
        std_df=df[colu].std()
        df_out= df[(df[colu] <= 3*std_df+mean_df) & (df[colu] >= (mean_df-3*std_df))]
        df=df_out
        i=i+1
        
    return df_out

total_wine=outliers_idr(total_wine)
total_wine= total_wine.set_index(np.arange(len(total_wine)))



In [206]:
#encoding
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
#le0=OneHotEncoder(sparse=False)
#total_wine['wine_type']=le0.fit_transform(total_wine[['wine_type']])
#le0.inverse_transform(total_wine['wine_type'].unique()) ####?
total_wine['wine_type']=total_wine['wine_type'].apply(lambda x: 0 if 'w' else 1)

le1=LabelEncoder()
total_wine['quality_label']=le1.fit_transform(total_wine['quality_label'])
print(le1.inverse_transform(total_wine['quality_label'].unique())) ###???

['medium' 'low' 'high']


In [207]:
# ML for quality of wine Decision tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score


model=DecisionTreeClassifier()
x=total_wine.drop(columns='quality_label')
y=total_wine['quality_label']
scale=MinMaxScaler()
x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=0.3, random_state=0)
x_train_n=scale.fit_transform(x_train)
x_train_n=pd.DataFrame(x_train_n, columns=x_train.columns)
x_test_n=scale.fit_transform(x_test)
x_test_n=pd.DataFrame(x_test_n, columns=x_test.columns)
model.fit(x_train_n,y_train)
predict=model.predict(x_test_n)
print(y_test, predict)
score=accuracy_score(y_test, predict)
print('accuracy is: ', score)
print("Confusion matrix:")
print(confusion_matrix(y_test, predict))
print(classification_report(y_test,predict))
cohen=cohen_kappa_score(y_test, predict)
print('cohen_kappa is: ', cohen)

1386    2
5486    1
2724    2
1831    2
737     2
       ..
1475    2
831     2
1273    1
3718    2
3229    2
Name: quality_label, Length: 1776, dtype: int64 [2 1 2 ... 1 2 2]
accuracy is:  1.0
Confusion matrix:
[[  49    0    0]
 [   0  680    0]
 [   0    0 1047]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        49
           1       1.00      1.00      1.00       680
           2       1.00      1.00      1.00      1047

    accuracy                           1.00      1776
   macro avg       1.00      1.00      1.00      1776
weighted avg       1.00      1.00      1.00      1776

cohen_kappa is:  1.0


In [208]:
# ML for quality of wine  RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


model= RandomForestClassifier()
x=total_wine.drop(columns='quality_label')
y=total_wine['quality_label']
scale=MinMaxScaler()
x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=0.3, random_state=0)
x_train_n=scale.fit_transform(x_train)
x_train_n=pd.DataFrame(x_train_n, columns=x_train.columns)
x_test_n=scale.fit_transform(x_test)
x_test_n=pd.DataFrame(x_test_n, columns=x_test.columns)
model.fit(x_train_n,y_train)
predict=model.predict(x_test_n)
print(y_test, predict)
score=accuracy_score(y_test, predict)
print('accuracy is: ', score)
print("Confusion matrix:")
print(confusion_matrix(y_test, predict))
print(classification_report(y_test,predict))
cohen=cohen_kappa_score(y_test, predict)
print('cohen_kappa is: ', cohen)

1386    2
5486    1
2724    2
1831    2
737     2
       ..
1475    2
831     2
1273    1
3718    2
3229    2
Name: quality_label, Length: 1776, dtype: int64 [2 1 2 ... 1 2 2]
accuracy is:  0.9994369369369369
Confusion matrix:
[[  48    0    1]
 [   0  680    0]
 [   0    0 1047]]
              precision    recall  f1-score   support

           0       1.00      0.98      0.99        49
           1       1.00      1.00      1.00       680
           2       1.00      1.00      1.00      1047

    accuracy                           1.00      1776
   macro avg       1.00      0.99      1.00      1776
weighted avg       1.00      1.00      1.00      1776

cohen_kappa is:  0.9988845398224308


In [209]:
total_wine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5919 entries, 0 to 5918
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed_acidity         5919 non-null   float64
 1   volatile_acidity      5919 non-null   float64
 2   citric_acid           5919 non-null   float64
 3   residual_sugar        5919 non-null   float64
 4   chlorides             5919 non-null   float64
 5   free_sulfur_dioxide   5919 non-null   float64
 6   total_sulfur_dioxide  5919 non-null   float64
 7   density               5919 non-null   float64
 8   pH                    5919 non-null   float64
 9   sulphates             5919 non-null   float64
 10  alcohol               5919 non-null   float64
 11  quality               5919 non-null   int64  
 12  wine_type             5919 non-null   int64  
 13  quality_label         5919 non-null   int64  
dtypes: float64(11), int64(3)
memory usage: 693.6 KB


In [210]:
# ML for quality of wine  LinearDiscriminantAnalysis

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


model= LinearDiscriminantAnalysis()
x=total_wine.drop(columns='quality_label')
y=total_wine['quality_label']
scale=MinMaxScaler()
x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=0.3, random_state=0)
x_train_n=scale.fit_transform(x_train)
x_train_n=pd.DataFrame(x_train_n, columns=x_train.columns)
x_test_n=scale.fit_transform(x_test)
x_test_n=pd.DataFrame(x_test_n, columns=x_test.columns)
model.fit(x_train_n,y_train)
predict=model.predict(x_test_n)
print(y_test, predict)
score=accuracy_score(y_test, predict)
print('accuracy is: ', score)
print("Confusion matrix:")
print(confusion_matrix(y_test, predict))
print(classification_report(y_test,predict))
cohen=cohen_kappa_score(y_test, predict)
print('cohen_kappa is: ', cohen)

1386    2
5486    1
2724    2
1831    2
737     2
       ..
1475    2
831     2
1273    1
3718    2
3229    2
Name: quality_label, Length: 1776, dtype: int64 [2 1 2 ... 1 2 2]
accuracy is:  1.0
Confusion matrix:
[[  49    0    0]
 [   0  680    0]
 [   0    0 1047]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        49
           1       1.00      1.00      1.00       680
           2       1.00      1.00      1.00      1047

    accuracy                           1.00      1776
   macro avg       1.00      1.00      1.00      1776
weighted avg       1.00      1.00      1.00      1776

cohen_kappa is:  1.0
